This notebook contains a few examples for bcc Fe to illustrate how VASP_job works

- Note: They only show how to create the inputs (set_calculation()), to submit the job, use:
   
   self.run_vasp()

In [1]:
import os
from subprocess import run
import numpy as np
from VASP_job.code.main import VASP_job

import pandas as pd
import ase
from ase.spacegroup import crystal

# Define a custom formatting function for vectors in pandas
format_vector = lambda vector: '[' + ', '.join(['{:.10f}'.format(x) for x in vector]) + ']'

import nglview

In [2]:
dir_0 = os.getcwd()

In [3]:
# set executable
executable_path= "/home/emendive/workbench/work/from_claix/codes/VASP/vasp.5.4.4-flag4/bin"
potential_path = "/home/emendive/workbench/work/from_claix/codes/VASP/potentials/potpaw_PBE"

# FM bcc Fe (2x1x1 supercell)

set working directory

In [4]:
cwd = dir_0 + "/FM_2x1x1"
if not os.path.exists(cwd):
   run("mkdir " + cwd, shell=True)

set crystal structure

In [5]:
a = 2.79 # A
alpha = 90
structure = crystal(('Fe'),
                       basis=[(0.00, 0.00, 0.00)],
                       spacegroup=229,
                       cellpar=[a, a, a, alpha, alpha, alpha])

set calculation

In [6]:
# create vasp class
vasp = VASP_job(verbose="low", executable_path=executable_path, potential_path=potential_path)

# update current working directory
vasp.io.cwd = cwd

In [7]:
# set magdirs for a FM state
number_of_atoms = len(structure)
magdirs   = []
for i in range(number_of_atoms):
   if 'Fe' == structure.get_chemical_symbols()[i]:
      magdirs.append([0, 0, 1])

structure.new_array("magdirs", magdirs, dtype=float)

In [8]:
# SUPERCELL (should be done after adding arrays):
structure = structure.repeat((2,1,1))
# update kpoints
vasp.structure.kpoints = "6 12 12"

In [9]:
# set calculation, which creates all input files
vasp.set_calculation(structure)

visualize structure

In [10]:
view = nglview.show_ase(structure)
view.add_unitcell()
view

NGLWidget()

visualize some inputs

In [11]:
vasp.df

,elements,positions,magdirs,ms,betahs,magmoms,B_CONSTRs
0,Fe,"[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]",1.0,inf,"[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]"
1,Fe,"[1.395, 1.395, 1.395]","[0.0, 0.0, 1.0]",1.0,inf,"[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]"
2,Fe,"[2.79, 0.0, 0.0]","[0.0, 0.0, 1.0]",1.0,inf,"[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]"
3,Fe,"[4.1850000000000005, 1.395, 1.395]","[0.0, 0.0, 1.0]",1.0,inf,"[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]"


In [12]:
df_magmoms = vasp.df["magmoms"].apply(format_vector)
df_magmoms

0    [0.0000000000, 0.0000000000, 1.0000000000]
1    [0.0000000000, 0.0000000000, 1.0000000000]
2    [0.0000000000, 0.0000000000, 1.0000000000]
3    [0.0000000000, 0.0000000000, 1.0000000000]
Name: magmoms, dtype: object

In [13]:
print("kpoints =",vasp.structure.kpoints)
print("\nlattice vectors =\n",vasp.structure.lattice_vectors)
print("\nspecies =",vasp.structure.species)
print("\nelements =",vasp.structure.elements)

kpoints = 6 12 12

lattice vectors =
 [[5.58 0.   0.  ]
 [0.   2.79 0.  ]
 [0.   0.   2.79]]

species = ['Fe']

elements = 0    Fe
1    Fe
2    Fe
3    Fe
Name: elements, dtype: object
